### Launch EVA DB
Run the command `python eva.py` in the server where you want to deploy EVA

### Establish Connection With Eva DB

In [22]:
import sys
sys.path.insert(0,'..')
from src.server.db_api import connect

In [23]:
# import nest_asyncio
# nest_asyncio.apply()
connection = connect(host = '0.0.0.0', port = 5432) # hostname, port of the server where EVADB is running

In [24]:
cursor = connection.cursor()

### Upload the survellience video to analyse

In [25]:
cursor.execute('UPLOAD INFILE "../data/ua_detrac/ua_detrac.mp4" PATH "test_video.mp4";')
response = cursor.fetch_one()
print(response)

Response Object:
@status: -1
@batch: Batch Object:
@dataframe:                                                error
0  [Errno 13] Permission denied: '/tmp/test_video...
@batch_size: 1
@identifier_column: id
@metrics: None


### Load video into EVA

In [37]:
cursor.execute('LOAD DATA INFILE "test_video.mp4" INTO MyVideo;')
response = cursor.fetch_one()
print(response)

Response Object:
@status: -1
@batch: Batch Object:
@dataframe:                      error
0  list index out of range
@batch_size: 1
@identifier_column: id
@metrics: None


### Optional - Register FasterRCNN (object detection) model into EVA 

In [28]:
cursor.execute("""CREATE UDF FastRCNNObjectDetector 
                  INPUT  (frame NDARRAY UINT8(3, 256, 256))
                  OUTPUT (labels NDARRAY STR(20), bboxes NDARRAY INT32(20, 4))
                  TYPE  Classification
                  IMPL  'src/udfs/fastrcnn_object_detector.py'""")
response = cursor.fetch_one()
print(response)

Response Object:
@status: -1
@batch: Batch Object:
@dataframe:                                               error
0  Exception occurred while committing to database.
@batch_size: 1
@identifier_column: id
@metrics: None


### Run Object detector on the video

In [35]:
cursor.execute("""SELECT FastRCNNObjectDetector(data) FROM MyVideo WHERE id < 10""")
response = cursor.fetch_one()
print(response)

Response Object:
@status: -1
@batch: Batch Object:
@dataframe:                                               error
0  Exception occurred while committing to database.
@batch_size: 1
@identifier_column: id
@metrics: None
